In [1]:
import torch

torch.cuda.is_available()

/home/rame/anaconda3/envs/nlp/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
%load_ext autoreload
%autoreload 2
import sys

In [3]:
sys.path.append("/home/rame/trl/examples/llama")

In [4]:
from llama_inference_imdb import *


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues


In [5]:
script_args = DefaultArgs()
assert "llama" in script_args.base_model_name.lower()
print(f"Load LMs with {script_args.peft_names}")
instructions = Instructions(prompt=script_args.prompt)

sentiment_pipes = Pipelines.load_pipes(script_args.sentiment_models)

Load LMs with [<llama_experiments.Experiments object at 0x7f2ec9b0edf0>]
Load sentiment model with ['OpenAssistant/reward-model-deberta-v3-base', 'ChaiML/gpt2_base_retry_and_continue_5m_reward_model', 'ChaiML/gpt2_base_retry_and_continue_12m_reward_model', 'sugam11/gpt2-rlhf-reward', 'theblackcat102/electra-large-reward-model']


In [6]:
instructions.instruction_movie = "Generate a movie review."

In [7]:
responses_text = ["Isaac Florentine has made some of the most entertaining and action-packed movies of the last decade. His latest"]

In [8]:
sentiment_pipes[-1].model.name_or_path

'theblackcat102/electra-large-reward-model'

In [9]:
sent_kwargs = {"return_all_scores": True, "function_to_apply": "none", "batch_size": 1}

rewards = [
    [
            llama_utils.transform_text(
                sentiment_pipe, response_text, instruction=instructions.instruction_movie
            ) for sentiment_pipe in sentiment_pipes
    ] for response_text in responses_text
]

In [10]:
rewards

[['[CLS]Generate a movie review.[SEP]Isaac Florentine has made some of the most entertaining and action-packed movies of the last decade. His latest',
  'User: Generate a movie review.\nBot: Isaac Florentine has made some of the most entertaining and action-packed movies of the last decade. His latest',
  'User: Generate a movie review.\nBot: Isaac Florentine has made some of the most entertaining and action-packed movies of the last decade. His latest',
  'Instruction: Generate a movie review. Response: Isaac Florentine has made some of the most entertaining and action-packed movies of the last decade. His latest',
  'Instruction: Generate a movie review. Response: Isaac Florentine has made some of the most entertaining and action-packed movies of the last decade. His latest']]

In [11]:
sent_kwargs = {"return_all_scores": True, "function_to_apply": "none", "batch_size": 1}

rewards = [
    [
        sentiment_pipe(
            llama_utils.transform_text(
                sentiment_pipe, response_text, instruction=instructions.instruction_movie
            ), **sent_kwargs
        ) for sentiment_pipe in sentiment_pipes
    ] for response_text in responses_text
]

/home/rame/anaconda3/envs/nlp/lib/python3.8/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [12]:
rewards

[[[[{'label': 'REWARD', 'score': -2.8863346576690674}]],
  [[{'label': 'LABEL_0', 'score': -1.265516996383667},
    {'label': 'LABEL_1', 'score': 0.5113178491592407}]],
  [[{'label': 'LABEL_0', 'score': -0.8205734491348267},
    {'label': 'LABEL_1', 'score': 0.5476537942886353}]],
  [[{'label': 'LABEL_0', 'score': -0.1741766482591629}]],
  [[{'label': 'LABEL_0', 'score': -2.5120272636413574}]]]]

In [ ]:
rewards = [
    [
        apply_sentiment_pipe(sentiment_pipe, response_text)
        for sentiment_pipe in sentiment_pipes
    ]
    for response_text in responses_text
]

In [ ]:
rewards

In [ ]:
rewards

In [ ]:
# 1. load all key components
base_model = Loader.load_base_model(script_args.base_model_name)
tokenizer = Tokenizer.load_tokenizer(script_args.base_model_name)

In [ ]:
if False:
    parser = HfArgumentParser(ScriptArguments)
    script_args = parser.parse_args_into_dataclasses()[0]
else:
    script_args = NotebookArgs()

In [ ]:
base_model = Loader.load_base_model(script_args.model_name)

In [ ]:
model, lora_model = Loader.load_peft_model(base_model, lora_config="nolora")

In [ ]:
model.is_peft_model

In [ ]:
lora_model.is_peft_model

In [ ]:
script_args = ScriptArguments.notebook_get_args()
assert "llama" in script_args.base_model_name.lower()
print(f"Load LMs with {script_args.peft_names}")

In [ ]:
base_model = Loader.load_base_model(script_args.base_model_name)

In [ ]:
sd = base_model.state_dict()

In [ ]:
sd = base_model_v2.state_dict()

In [ ]:
sd["model.layers.2.self_attn.rotary_emb.inv_freq"]

In [ ]:
base_model_v2 = LlamaForCausalLM.from_pretrained(
            script_args.peft_names[0],     return_dict=True,
    torch_dtype=torch.float16,
        )

In [ ]:
device = 0 if torch.cuda.is_available() else "cpu"
print(f"Load LMs with {script_args.model_names}")
dict_models_to_merge = OrderedDict({model_name: load_model(model_name) for model_name in script_args.model_names})

tokenizer = AutoTokenizer.from_pretrained(
    PeftConfig.from_pretrained(script_args.model_names[0]).base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

print(f"Load sentiment model with {script_args.sentiment_models}")
sentiment_pipes = [
    pipeline("sentiment-analysis", model=sentiment_model, device=device)
    for sentiment_model in script_args.sentiment_models]

list_states_dict = []
for current_model in dict_models_to_merge.values():
    current_weights = copy.deepcopy(current_model.state_dict())
    list_states_dict.append(current_weights)

In [ ]:
samples_query_tensors = get_samples_query_tensors()

In [ ]:
wa = enrich_wa_states(list_states_dict, coefficients=[0.3, 0.7])
list_rewards_wa_samples = predict({"wa": wa}, samples_query_tensors, verbose=True)

In [ ]:
wa = enrich_wa_states(list_states_dict, coefficients=[0.25, 0.75])
list_rewards_wa_samples = predict({"wa": wa}, samples_query_tensors, verbose=True)

In [ ]:

for coeff in [x / 20 for x in range(17, -1, -1)]:
    wa = enrich_wa_states(list_states_dict, coefficients=[1 - coeff, coeff])
    list_rewards_wa_samples = predict({"wa": wa}, samples_query_tensors)
    print(coeff)
    print(list_rewards_wa_samples)
    print("\n")